In [ ]:
!pip install translate-toolkit transformers sentencepiece


In [ ]:
!wget -O dataset.txt.zip https://opus.nlpl.eu/download.php?f=TEP/v1/moses/en-fa.txt.zip
!unzip dataset.txt.zip

In [3]:
from IPython.display import display
from IPython.html import widgets
import matplotlib.pyplot as plt
import numpy as np
import random
import seaborn as sns
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook

sns.set()

/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:100,garbage_collection_threshold:0.8

In [6]:

with open("TEP.en-fa.en", 'rb') as enfile:
  en_lines_list = [line.decode("UTF-8").rstrip() for line in enfile]

with open('TEP.en-fa.fa') as fafile:
  fa_lines_list = [line.rstrip() for line in fafile]

# {"en": "hello", "fa": "سلام"}
sentences = []
for i in range(0, len(en_lines_list)):
  sentences.append({
        "en": en_lines_list[i],
        "fa": fa_lines_list[i]
  })

print("len en lines: ", len(en_lines_list))
print("len fa lines: ", len(fa_lines_list))
print("first en sentence: ", en_lines_list[0])
print("first fa sentence: ", fa_lines_list[0])

len en lines:  612086
len fa lines:  612086
first en sentence:  raspy breathing .
first fa sentence:  صداي خر خر .


In [7]:
sentences = sentences[:]
sentences[2:8]

[{'en': 'maybe its the wind .', 'fa': 'شايد صداي باد باشه .'},
 {'en': 'no .', 'fa': 'نه .'},
 {'en': 'stop please stop .',
  'fa': 'دست نگه داريد خواهش ميکنم دست نگه داريد .'},
 {'en': 'you have a week , evans then well burn the house .',
  'fa': 'اوانز تو فقط يک هفته وقت داري وگرنه خونتو خواهيم سوزوند .'},
 {'en': 'william .', 'fa': 'ويليام .'},
 {'en': 'god damn it , william .', 'fa': 'لعنتي . ويليام 8 .'}]

In [8]:
random.shuffle(sentences)
train_slice = int(len(sentences) * 0.80)
train_dataset = sentences[:train_slice]
test_dataset = sentences[train_slice:]

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Prepare data for feed to model

In [5]:
LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'fa': '<fa>',
}

In [6]:
def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]
  
def encode_target_str(text, tokenizer, seq_len,
                      lang_token_map=LANG_TOKEN_MAPPING):
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)
  
  return token_ids[0]

def format_translation_data(translations, input_lang, target_lang,
                            tokenizer, seq_len=40):

  # Get the translations for the batch
  input_text = translations[input_lang]
  target_text = translations[target_lang]

  if input_text is None or target_text is None:
    return None

  input_token_ids = encode_input_str(
      input_text, target_lang, tokenizer, seq_len, LANG_TOKEN_MAPPING)
  
  target_token_ids = encode_target_str(
      target_text, tokenizer, seq_len, LANG_TOKEN_MAPPING)

  return input_token_ids, target_token_ids

def transform_batch(batch, input_lang, target_lang, tokenizer):
  inputs = []
  targets = []
  for translation_set in batch:
    formatted_data = format_translation_data(
        translation_set, input_lang, target_lang, tokenizer, max_seq_len)
    
    if formatted_data is None:
      continue
    
    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))
    
  batch_input_ids = torch.cat(inputs).cuda()
  batch_target_ids = torch.cat(targets).cuda()

  return batch_input_ids, batch_target_ids

def get_data_generator(dataset, input_lang, target_lang, tokenizer, batch_size=32):
  random.shuffle(dataset)
  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
    yield transform_batch(raw_batch, input_lang, target_lang, tokenizer)

# Load Tokenizer & Model English to Persian


In [ ]:
model_repo_en2fa = 'persiannlp/mt5-small-parsinlu-translation_en_fa'
model_path_en2fa = "/content/gdrive/MyDrive/mt5_translation_en2fa.pt"

In [ ]:
# Model description: https://metatext.io/models/persiannlp-mt5-small-parsinlu-translation_en_fa
model_en2fa = AutoModelForSeq2SeqLM.from_pretrained(model_repo_en2fa)
tokenizer_en2fa = AutoTokenizer.from_pretrained(model_repo_en2fa, use_fast=False)
model_en2fa = model_en2fa.cuda()
model_en2fa.config.max_length = 40
max_seq_len = model_en2fa.config.max_length

In [ ]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer_en2fa.add_special_tokens(special_tokens_dict)
model_en2fa.resize_token_embeddings(len(tokenizer_en2fa))

In [ ]:
# Testing `data_transform`
in_ids, out_ids = format_translation_data(
    train_dataset[9], "en", "fa", tokenizer_en2fa)

print(' '.join(tokenizer_en2fa.convert_ids_to_tokens(in_ids)))
print(' '.join(tokenizer_en2fa.convert_ids_to_tokens(out_ids)))

In [ ]:
model_en2fa.load_state_dict(torch.load(model_path_en2fa))

In [ ]:
# Constants
n_epochs = 2
batch_size = 32
print_freq = 100
checkpoint_freq = 1000
lr = 5e-4
n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)

# Optimizer
optimizer = optim.AdamW(model_en2fa.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)

In [ ]:
def eval_model(model, gdataset, max_iters=8):
  test_generator = get_data_generator(gdataset, "en", "fa",
                                      tokenizer_en2fa, batch_size)
  eval_losses = []
  for i, (input_batch, label_batch) in enumerate(test_generator):
    if i >= max_iters:
      break

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)
    eval_losses.append(model_out.loss.item())

  return np.mean(eval_losses)

In [ ]:
losses_en2fa = []
best_loss = 1000

In [ ]:
for epoch_idx in range(n_epochs):
  # Randomize data order
  data_generator = get_data_generator(train_dataset, "en", "fa",
                                      tokenizer_en2fa, batch_size)
                
  for batch_idx, (input_batch, label_batch) \
      in tqdm_notebook(enumerate(data_generator), total=n_batches):
    optimizer.zero_grad()

    # Forward pass
    model_out = model_en2fa.forward(
        input_ids = input_batch,
        labels = label_batch)

    # Calculate loss and update weights
    loss = model_out.loss
    losses_en2fa.append(loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Print training update info
    if (batch_idx + 1) % print_freq == 0:
      avg_loss = np.mean(losses_en2fa[-print_freq:])
      print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
          epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))
      
    if (batch_idx + 1) % checkpoint_freq == 0:
      test_loss = eval_model(model_en2fa, test_dataset)
      print('Test loss: {:.3f}'.format(test_loss))
      # dont have overfitting
      if abs(test_loss - avg_loss) < 1:
        if test_loss < best_loss:
          best_loss = test_loss
          print('Saving model with test loss of {:.3f}'.format(test_loss))
          torch.save(model_en2fa.state_dict(), model_path_en2fa)


In [ ]:
# Graph the loss

window_size = 50
smoothed_losses = []
for i in range(len(losses_en2fa)-window_size):
  smoothed_losses.append(np.mean(losses_en2fa[i:i+window_size]))

plt.plot(smoothed_losses[100:])


# Load Tokenizer & Model Persian to English

In [ ]:
model_repo_fa2en = 'google/mt5-small'
model_path_fa2en = "/content/gdrive/MyDrive//mt5_translation_fa2en.pt"

In [ ]:
# Model description: https://huggingface.co/google/mt5-small
tokenizer_fa2en = AutoTokenizer.from_pretrained(model_repo_fa2en, use_fast=False)
model_fa2en = AutoModelForSeq2SeqLM.from_pretrained(model_repo_fa2en)
model_fa2en = model_fa2en.cuda()
model_fa2en.config.max_length = 40
max_seq_len = model_fa2en.config.max_length

In [ ]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer_fa2en.add_special_tokens(special_tokens_dict)
model_fa2en.resize_token_embeddings(len(tokenizer_fa2en))

In [ ]:
# Testing `data_transform`
in_ids, out_ids = format_translation_data(
    train_dataset[1], "fa", "en", tokenizer_fa2en)

print('|'.join(tokenizer_fa2en.convert_ids_to_tokens(in_ids)))
print('|'.join(tokenizer_fa2en.convert_ids_to_tokens(out_ids)))

In [ ]:
model_fa2en.load_state_dict(torch.load(model_path_fa2en))

In [ ]:
# Constants
n_epochs = 2
batch_size = 32
print_freq = 100
checkpoint_freq = 1000
lr = 5e-4
n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)

# Optimizer
optimizer = optim.AdamW(model_fa2en.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)

In [ ]:
def eval_model(model, gdataset, max_iters=8):
  test_generator = get_data_generator(gdataset, "fa", "en",
                                      tokenizer_fa2en, batch_size)
  eval_losses = []
  for i, (input_batch, label_batch) in enumerate(test_generator):
    if i >= max_iters:
      break

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)
    eval_losses.append(model_out.loss.item())

  return np.mean(eval_losses)

In [ ]:
losses_fa2en = []
best_loss = 1000

In [ ]:
for epoch_idx in range(n_epochs):
  # Randomize data order
  data_generator = get_data_generator(train_dataset, "fa", "en",
                                      tokenizer_fa2en, batch_size)
                
  for batch_idx, (input_batch, label_batch) \
      in tqdm_notebook(enumerate(data_generator), total=n_batches):
    optimizer.zero_grad()

    # Forward pass
    model_out = model_fa2en.forward(
        input_ids = input_batch,
        labels = label_batch)

    # Calculate loss and update weights
    loss = model_out.loss
    losses_fa2en.append(loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Print training update info
    if (batch_idx + 1) % print_freq == 0:
      avg_loss = np.mean(losses_fa2en[-print_freq:])
      print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
          epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))
      
    if (batch_idx + 1) % checkpoint_freq == 0:
      test_loss = eval_model(model_fa2en, test_dataset)
      print('Test loss: {:.3f}'.format(test_loss))
      # dont have overfitting
      if abs(test_loss - avg_loss) < 1:
        if test_loss < best_loss:
          best_loss = test_loss
          print('Saving model with test loss of {:.3f}'.format(test_loss))
          torch.save(model_fa2en.state_dict(), model_path_fa2en)


In [ ]:
# Graph the loss

window_size = 50
smoothed_losses = []
for i in range(len(losses_fa2en)-window_size):
  smoothed_losses.append(np.mean(losses_fa2en[i:i+window_size]))

plt.plot(smoothed_losses[100:])

In [ ]:
#@title English To Persian Translation
input_text = 'I am new to work as a translator. please ask me simple sentences' #@param {type:"string"}
output_language = 'fa' #@param ["fa"]
input_ids = encode_input_str(
    text = input_text,
    target_lang = output_language,
    tokenizer = tokenizer_en2fa,
    seq_len = model_en2fa.config.max_length,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cuda()

output_tokens = model_en2fa.generate(input_ids, num_beams=20, length_penalty=0.2)
print(input_text + '  ->  ' + \
      tokenizer_en2fa.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
#@title Persian To English Translation
input_text = 'امروز درس بزرگی یاد گرفته ام دوست دارم امشب توی دفترچه ام یادداشت اش کنم.' #@param {type:"string"}
output_language = 'en' #@param ["en"]

input_ids = encode_input_str(
    text = input_text,
    target_lang = output_language,
    tokenizer = tokenizer_fa2en,
    seq_len = model_fa2en.config.max_length,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cuda()

output_tokens = model_fa2en.generate(input_ids, num_beams=20, length_penalty=0.2)
print(input_text + '  ->  ' + \
      tokenizer_fa2en.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
!curl bashupload.com -T ./mt5_translation_fa2en.pt

In [ ]:
def translate(texts, output_language):
  if output_language == 'fa':
    tokenizer = tokenizer_en2fa
    model = model_en2fa
  elif output_language == 'en':
    tokenizer = tokenizer_fa2en
    model = model_fa2en
  for input_text in texts:
    input_ids = encode_input_str(
    text=input_text,
    target_lang=output_language,
    tokenizer=tokenizer,
    seq_len=model.config.max_length,
    lang_token_map=LANG_TOKEN_MAPPING)
    input_ids = input_ids.unsqueeze(0).cuda()
    output_tokens = model.generate(input_ids, num_beams=20, length_penalty=0.2)
    print(input_text + '  ->  ' + \
        tokenizer.decode(output_tokens[0], skip_special_tokens=True))


In [ ]:

#@title 10 text translation
text1 = "I am new to work as a translator. please ask me simple sentences" #@param {type:"string"}
text2 = "i learned a great lesson today. i like to write it tonight in my notebook." #@param {type:"string"}
text3 = "Some stars appeared in the night sky." #@param {type:"string"}
text4 = "We would often discuss our future." #@param {type:"string"}
text5 = "He sells foodstuffs, such as butter, cheese, and sugar." #@param {type:"string"}
text6 = "It would be so cool if I could speak ten languages!" #@param {type:"string"}
text7 = "The grade seven and the grade eight students both have completely separate timetables." #@param {type:"string"}
text8 = "We've been married for five years." #@param {type:"string"}
text9 = "" #@param {type:"string"}
text10 = "" #@param {type:"string"}
output_language = 'en' #@param ["en", "fa"]

text_dic = {
    1: text1, 2: text2, 3: text3, 4: text4, 5: text5,
    6: text6, 7: text7, 8: text8, 9: text9, 10: text10
}
texts = []
for i in text_dic.keys():
  if text_dic[i] != "":
    texts.append(text_dic[i])

translate(texts, output_language)

In [17]:
def translate(texts, output_language):
  if output_language == 'fa':
    tokenizer = tokenizer_en2fa
    model = model_en2fa
  elif output_language == 'en':
    tokenizer = tokenizer_fa2en
    model = model_fa2en
  for input_text in texts:
    input_ids = encode_input_str(
    text=input_text,
    target_lang=output_language,
    tokenizer=tokenizer,
    seq_len=model.config.max_length,
    lang_token_map=LANG_TOKEN_MAPPING)
    input_ids = input_ids.unsqueeze(0).cuda()
    output_tokens = model.generate(input_ids, num_beams=20, length_penalty=0.2)
    print(input_text + '  ->  ' + \
        tokenizer.decode(output_tokens[0], skip_special_tokens=True))


In [38]:

#@title 10 text translation
text1 = "I am new to work as a translator. please ask me simple sentences" #@param {type:"string"}
text2 = "i learned a great lesson today. i like to write it tonight in my notebook." #@param {type:"string"}
text3 = "Some stars appeared in the night sky." #@param {type:"string"}
text4 = "We would often discuss our future." #@param {type:"string"}
text5 = "He sells foodstuffs, such as butter, cheese, and sugar." #@param {type:"string"}
text6 = "It would be so cool if I could speak ten languages!" #@param {type:"string"}
text7 = "The grade seven and the grade eight students both have completely separate timetables." #@param {type:"string"}
text8 = "We've been married for five years." #@param {type:"string"}
text9 = "" #@param {type:"string"}
text10 = "" #@param {type:"string"}
output_language = 'en' #@param ["en", "fa"]

text_dic = {
    1: text1, 2: text2, 3: text3, 4: text4, 5: text5,
    6: text6, 7: text7, 8: text8, 9: text9, 10: text10
}
texts = []
for i in text_dic.keys():
  if text_dic[i] != "":
    texts.append(text_dic[i])

translate(texts, output_language)

I am new to work as a translator. please ask me simple sentences  ->  i am new to work as translator. please ask me simple sentences.
i learned a great lesson today. i like to write it tonight in my notebook.  ->  i learned a great lesson today. i like to write it tonight in my notebook.
Some stars appeared in the night sky.  ->  some stars in the night sky.
We would often discuss our future.  ->  we would often discuss our future.
He sells foodstuffs, such as butter, cheese, and sugar.  ->  he sells food, butter butter cheese and sugar.
It would be so cool if I could speak ten languages!  ->  it would be cool if i could speak tenlanguages!
The grade seven and the grade eight students both have completely separate timetables.  ->  grade seven and grade eight students both have separate tables.
We've been married for five years.  ->  we've been married for five years for five years.
